In [ ]:
#!/usr/bin/env


"""


make line datacubes and integrated maps
and compare applying continuum selfcal
NOTE: this is intended to be an interactive, iterative process
      so this is more a log that should be run by cutting and
      pasting into casa rather than as an executable script
      search "CHANGEME" for variables to be changed
10/9/15 MCA
"""


#  Setup 

LupusI_15  +15:45:12.86 -34:17:30.8
Class II, K2


In [ ]:
field = 10                                                  # CHANGEME 
fitspw    = '2,3:0~2049;2151~2899;3051~3480,4'              # line-free channels for fitting continuum
fitspw    = '2,3,4'                                         # line-free channels for fitting continuum
linespw   = '0,1,3'                                         # line spectral windows (C18O, 13CO, CN)
robust     = 0.5                                            # CHANGEME
imsize     = [640,640]
cell       = '0.03arcsec'
imagermode = 'csclean'
outframe   = 'lsrk'
veltype    = 'radio'
width      = '1.0km/s'
start      = '-3km/s'
nchan      = 16
xc         = 329                               # CHANGEME
yc         = 314                               # CHANGEME
in_a       = 80
out_a      = 120
aper       = 0.5
boxwidth = 300.
box = rg.box([xc-boxwidth,yc-boxwidth],[xc+boxwidth,yc+boxwidth])


#  Create continuum subtracted line datasets 

In [ ]:
uvcontsub(vis       = 'f'+str(field)+'.vis',                # full vis file for this field
          spw       = linespw,                              # line spw (for cont subtraction)
          fitspw    = fitspw,                               # cont spw
          combine   = 'spw',
          solint    = 'int',
          fitorder  = 1,
          want_cont = False)                                # should not be changed.


#  13CO line 

first try on 13CO line as thats going to be the brightest
os.system('rm -rf f'+str(field)+'_13co32*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_13co32',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '330.58797GHz',
      niter         = 500,
      threshold     = 0,
      interactive   = False,
      mask          = 'f10_cont_mask.crtf',
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


could not see line, so just cleaned lightly 
within continuum region for all channels at once
use viewer to check channel maps and spectrum
make sure that velocity range is adequate and continuum subtraction ok


In [ ]:
imview(raster   = [{'file':'f'+str(field)+'_13co32.image'}],
        contour = [{'file':'f'+str(field)+'_cont.fits'}])


export cube to fits file    


In [ ]:
fbase = 'f'+str(field)+'_13co32'
os.system('rm -rf '+fbase+'.cube.fits')
exportfits(imagename=fbase+'.image',fitsimage=fbase+'.cube.fits')


redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_13co32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_13co32.image',            # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_13co32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[1km/s,9km/s]'))                        # CHANGEME (based on above analysis!!)


difficult to see peaks but this gives best SNR
export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_13co32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux
imview(raster=[{'file':'f'+str(field)+'_13co32.mom0'}])


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_13co32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_13co32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 893.94 mJy, rms = 65.27 mJy, S/N = 13.7
view continuum and gas


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_13co32.mom0'}],
       contour = [{'file':'f'+str(field)+'_cont.fits'}])


re-center image on source and use measure.py to get COG flux


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_13co32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_13co32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_13co32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_13co32.mom0_cropped.fits')


In [ ]:
Measuring COG for G/f10_13co32.mom0_cropped.fits
Assuming object center (300.0,300.0)
Background: -3.57 mJy/beam km/s
RMS in annulus 4.0-6.0 arcsec = 67.85 mJy/beam km/s
   i   radius    flux      err       snr
       (asec)    (mJy)     (mJy)
   0     0.10   115.30    15.63      7.4
   1     0.20   347.64    46.50      7.5
   2     0.30   602.64    90.13      6.7
   3     0.40   791.08   102.06      7.8
   4     0.50   893.94   111.98      8.0
   5     0.60   915.36   132.82      6.9
   6     0.70   882.96   165.24      5.3
   7     0.80   836.34   208.50      4.0
F = 915.36 mJy
E = 132.82 mJy
S = 6.89
D = 1.20 arcsec


#  C18O line  

don't bother with selfcal as it doesn't help...
os.system('rm -rf f'+str(field)+'_c18o32*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_c18o32',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '329.33055GHz',
      niter         = 500,
      threshold     = 0,
      interactive   = False,
      mask          = 'f10_cont_mask.crtf',
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


could not see line, so just cleaned lightly 
within continuum region for all channels at once
export cube to fits file    


In [ ]:
fbase = 'f'+str(field)+'_c18o32'
os.system('rm -rf '+fbase+'.cube.fits')
exportfits(imagename=fbase+'.image',fitsimage=fbase+'.cube.fits')


use viewer to check channel maps and spectrum
make sure that velocity range is adequate and continuum subtraction ok


In [ ]:
imview(raster   = [{'file':'f'+str(field)+'_c18o32.image'}],
        contour = [{'file':'f'+str(field)+'_cont.fits'}])


redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_c18o32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_c18o32.image',            # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_c18o32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[1km/s,9km/s]'))                        # CHANGEME (based on above analysis!!)


can't see line so will use same velocity range as 13co32
export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_c18o32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_c18o32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_c18o32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 444.33 mJy, rms = 82.57 mJy, S/N = 5.4
view continuum and gas


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_c18o32.mom0'}],
       contour = [{'file':'f'+str(field)+'_cont.fits'}])


re-center image on source and use measure.py to get COG flux


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_c18o32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_c18o32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_c18o32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_c18o32.mom0_cropped.fits')


In [ ]:
Measuring COG for G/f10_c18o32.mom0_cropped.fits
Assuming object center (300.0,300.0)
Background: -2.17 mJy/beam km/s
RMS in annulus 4.0-6.0 arcsec = 79.16 mJy/beam km/s
   i   radius    flux      err       snr
       (asec)    (mJy)     (mJy)
   0     0.10    36.29    21.22      1.7
   1     0.20   117.10    46.61      2.5
   2     0.30   237.83    77.31      3.1
   3     0.40   355.40   106.28      3.3
   4     0.50   444.33   131.95      3.4
   5     0.60   422.48   155.66      2.7
   6     0.70   306.09   201.20      1.5
   7     0.80   203.48   178.11      1.1
F = 444.33 mJy
E = 131.95 mJy
S = 3.37
D = 1.00 arcsec


#  Image CN  

don't bother with selfcal as it doesn't help...
os.system('rm -rf f'+str(field)+'_cn32_b4sc*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_cn32_b4sc',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '340.24777GHz',
      niter         = 500,
      threshold     = 0,
      interactive   = False,
      mask          = 'f10_cont_mask.crtf',
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


could not see line, so just cleaned lightly 
within continuum region for all channels at once
use viewer to check channel maps and spectrum
make sure that velocity range is adequate and continuum subtraction ok


In [ ]:
imview(raster   = [{'file':'f'+str(field)+'_cn32_b4sc.image'}],
        contour = [{'file':'f'+str(field)+'_cont.fits'}])


redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_cn32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_cn32_b4sc.image',            # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_cn32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[2km/s,6km/s]'))                        # CHANGEME (based on above analysis!!)


can't see line 
export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_cn32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_cn32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_cn32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 198.15 mJy, rms = 35.83 mJy, S/N = 5.5
view continuum and gas


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cn32.mom0'}],
       contour = [{'file':'f'+str(field)+'_cont.fits'}])


re-center image on source and use measure.py to get COG flux


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_cn32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_cn32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_cn32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_cn32.mom0_cropped.fits')


In [ ]:
Measuring COG for G/f10_cn32.mom0_cropped.fits
Assuming object center (300.0,300.0)
Background: 0.40 mJy/beam km/s
RMS in annulus 4.0-9.0 arcsec = 36.83 mJy/beam km/s
   i   radius    flux      err
       (asec)    (mJy)     (mJy)
   0      0.1     -3.4      8.8
   1      0.2      4.3     25.1
   2      0.3     54.3     40.6
   3      0.4    129.3     52.3
   4      0.5    198.1     63.1
   5      0.6    231.3     82.7
   6      0.7    251.9    106.3
   7      0.8    273.3    117.9 <--- using this (flux levels off)
   8      0.9    280.2     99.8
   9      1.0    286.9    134.9
  10      1.1    308.1    156.4
  11      1.2    317.0    155.0
  12      1.3    295.8    162.4
  13      1.4    271.4    192.2
  14      1.5    258.8    229.6
F = 317.0 mJy
E = 155.0 mJy
D = 1.2 arcsec
